## Explore and cluster the neighborhoods in Toronto.

First importing the needed libaraies

In [1]:

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs
from geopy.geocoders import Nominatim 
import requests
import folium 

In [2]:
df_toronto_coor = pd.read_csv('toronto_df_coordinates.csv')

In [3]:
df_toronto_coor.head(5)

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Deleting the Unnamed 0 column

In [4]:
del df_toronto_coor["Unnamed: 0"]

In [14]:
#The latitude and longtitude of City of Toronto is obtained via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832


Generating the Map of Toronto

In [18]:
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

In [17]:
map_toronto

adding Markdowns to the Map

In [20]:
for lat, lng, borough, neighborhood in zip(df_toronto_coor['Latitude'], df_toronto_coor['Longitude'], df_toronto_coor['Borough'], df_toronto_coor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  

In [21]:
map_toronto

Saving the Map of toronto

In [22]:
map_toronto.save("my_map1.html" )

In [26]:

from IPython.display import HTML

HTML('<iframe src=my_map1.html width=700 height=450></iframe>')

C:\Users\Aresh\anaconda3\envs\tensorflow\lib\site-packages\IPython\core\display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


Starting queries to Foursquares

In [23]:
CLIENT_ID = '5JRSUMBYWQHHEZ3QT3QASLFTJKVSSBOHJEOCAX45ASNORIOF' # your Foursquare ID
CLIENT_SECRET = 'NIIEA2B4DM1SVMUSZ5LDBPSCZJXQY1ORBSOLVXSSZAKSTC0U' # your Foursquare Secret
ACCESS_TOKEN = 'ROOTVQZ3OZWFEYMIAGCVMZRYTZPU2TAWVMURN5P5FRBV3RMP' # your FourSquare Access Token
VERSION = '20180604'

# Creating the Map of North York

In [24]:
North_York = df_toronto_coor[df_toronto_coor['Borough'] == 'North York'].reset_index(drop=True)
North_York.head(7)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073
5,M3C,North York,Don Mills,43.725900,-79.340923
6,M2H,North York,Hillcrest Village,43.803762,-79.363452


In [25]:

address_nr = 'North York,Toronto'
latitude_nr = 43.7615
longitude_nr = -79.4111
print('The geograpical coordinate of North York are {}, {}.'.format(latitude_nr, longitude_nr))

The geograpical coordinate of North York are 43.7615, -79.4111.


In [28]:

map_north_york = folium.Map(location=[latitude_nr, longitude_nr], zoom_start=12)

# add markers to map
for lat, lng, label in zip(North_York['Latitude'], North_York['Longitude'], North_York['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_north_york)  

In [29]:
map_north_york.save('northyork_map.html')

In [30]:
from IPython.display import HTML

HTML('<iframe src=northyork_map.html width=700 height=450></iframe>')

# Get the 100 Venues in the Neighborhood "Victoria Park", from North York 

In [31]:
neighborhood_latitude = North_York.loc[1, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = North_York.loc[1, 'Longitude'] # neighbourhood longitude value

neighborhood_name = North_York.loc[1, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Victoria Village" are 43.7258823, -79.31557159999998.


In [32]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_nr, longitude_nr, VERSION, radius, LIMIT)

In [33]:
results = requests.get(url).json()


In [35]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

<ipython-input-36-9d4491af7edd>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Kinka Izakaya,Japanese Restaurant,43.760161,-79.409827
1,EatBKK,Thai Restaurant,43.759320,-79.410454
2,Longo’s Sheppard Centre,Grocery Store,43.762221,-79.410762
3,Sushi Moto Sake & Wine Bar,Sushi Restaurant,43.764095,-79.411344
4,Bake Code 烘焙密碼,Bakery,43.763969,-79.411641
5,Poke Eats,Poke Place,43.760060,-79.410862
6,Five Guys,Burger Joint,43.762400,-79.410763
7,MYMY Chicken,Fried Chicken Joint,43.764658,-79.411096
8,Whole Foods Market,Grocery Store,43.760662,-79.411044
9,Pizzaiolo,Pizza Place,43.764289,-79.411780


In [37]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

46 venues were returned by Foursquare.


In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get venues for each neighborhood in North York

In [39]:
NorthYork_venues = getNearbyVenues(names=North_York['Neighbourhood'],
                                   latitudes=North_York['Latitude'],
                                   longitudes=North_York['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [40]:
NorthYork_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
6,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
7,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
8,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Roots,43.718214,-79.463893,Boutique
9,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Kitchen Stuff Plus (Clearance Outlet),43.719096,-79.462675,Furniture / Home Store


In [41]:
NorthYork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Don Mills,25,25,25,25,25,25
Downsview,13,13,13,13,13,13
"Fairview, Henry Farm, Oriole",66,66,66,66,66,66
Glencairn,5,5,5,5,5,5
Hillcrest Village,5,5,5,5,5,5
Humber Summit,4,4,4,4,4,4


In [42]:

print('There are {} uniques categories.'.format(len(NorthYork_venues['Venue Category'].unique())))

There are 103 uniques categories.


In [43]:

# one hot encoding
northyork_onehot = pd.get_dummies(NorthYork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = NorthYork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped.head(7)

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.090909,...,0.0,0.045455,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,Bayview Village,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.250000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.038462,0.00,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.038462,0.076923,0.000000,0.000000,0.000000,0.0,0.000000
3,Don Mills,0.0,0.000000,0.000000,0.04,0.0,0.000000,0.040000,0.000000,0.000000,...,0.0,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,Downsview,0.0,0.076923,0.000000,0.00,0.0,0.000000,0.076923,0.000000,0.076923,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,"Fairview, Henry Farm, Oriole",0.0,0.000000,0.015152,0.00,0.0,0.015152,0.000000,0.030303,0.030303,...,0.0,0.000000,0.015152,0.000000,0.000000,0.015152,0.015152,0.015152,0.0,0.030303
6,Glencairn,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.200000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


Get top 10 venues per neighborhood

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [46]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Gas Station,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gift Shop,Diner,Grocery Store,Deli / Bodega
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Thai Restaurant,Italian Restaurant,Pizza Place,Sandwich Place,Butcher,Restaurant,Juice Bar,Café,Japanese Restaurant
3,Don Mills,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Sandwich Place,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café
4,Downsview,Grocery Store,Park,Bank,Athletics & Sports,Liquor Store,Shopping Mall,Baseball Field,Discount Store,Food Truck,Airport
5,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Shoe Store,Restaurant,Juice Bar,Japanese Restaurant,Bank,Bakery,Women's Store
6,Glencairn,Park,Pizza Place,Bakery,Pub,Japanese Restaurant,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
7,Hillcrest Village,Golf Course,Mediterranean Restaurant,Fast Food Restaurant,Pool,Dog Run,Women's Store,Dim Sum Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
8,Humber Summit,Furniture / Home Store,Pizza Place,Gym,Home Service,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
9,"Humberlea, Emery",Food Service,Baseball Field,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop


In [80]:

# import k-means from clustering stage
from sklearn.cluster import KMeans

index = [19,20,21,22, 23]
ny_data = North_York.drop(index)
# set number of clusters
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 3])

In [81]:
ny_merged = ny_data

# add clustering labels
ny_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

ny_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,French Restaurant,Hockey Arena,Coffee Shop,Pizza Place,Portuguese Restaurant,Intersection,Fast Food Restaurant,Deli / Bodega,Food Truck,Comfort Food Restaurant
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Boutique,Vietnamese Restaurant,Event Space,Coffee Shop,Miscellaneous Shop,Accessories Store,Arts & Crafts Store,Bakery
3,M3B,North York,Don Mills,43.745906,-79.352188,0,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Sandwich Place,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café
4,M6B,North York,Glencairn,43.709577,-79.445073,0,Park,Pizza Place,Bakery,Pub,Japanese Restaurant,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
5,M3C,North York,Don Mills,43.725900,-79.340923,0,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Sandwich Place,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café
6,M2H,North York,Hillcrest Village,43.803762,-79.363452,0,Golf Course,Mediterranean Restaurant,Fast Food Restaurant,Pool,Dog Run,Women's Store,Dim Sum Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
7,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,0,Coffee Shop,Bank,Gas Station,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gift Shop,Diner,Grocery Store,Deli / Bodega
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Shoe Store,Restaurant,Juice Bar,Japanese Restaurant,Bank,Bakery,Women's Store
9,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,3,Furniture / Home Store,Metro Station,Caribbean Restaurant,Massage Studio,Bar,Coffee Shop,Diner,Construction & Landscaping,Convenience Store,Cosmetics Shop


In [82]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_nr, longitude_nr], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighbourhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters.save('my_map3.html')

In [83]:
map_clusters